In [43]:
######################## Cleaning The Tweets ##########################
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
filename = 'train.csv'
df = pd.read_csv(filename)
stop = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

df['tweet'] = df['tweet'].apply(lambda x: p.clean(re.sub('#', '',x))) #Tweet Cleaner
df['tweet'] = df['tweet'].apply(lambda x: x.lower()) #Lower Casing
df['tweet'] = df['tweet'].str.replace('[^\w\s]','') #Removing Punctuations
df['tweet'] = df['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))#STopwords Removal
df['tweet'] = df['tweet'].apply(lambda x: " ".join([lemmatizer.lemmatize(word) for word in x.split()]))

df.to_csv('clean_'+str(filename))

In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import string

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

import preprocessor as p
from tqdm import tqdm

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [2]:
df = pd.read_csv('clean_train.csv')
df.head()

,id,label,tweet
0,1,0,father dysfunctional selfish drag kid dysfunct...
1,2,0,thanks lyft credit cant use cause dont offer w...
2,3,0,bihday majesty
3,4,0,model love u take u time ur
4,5,0,factsguide society motivation


In [3]:
df = df.dropna()


In [8]:
df['tweet'][3].split()

['model', 'love', 'u', 'take', 'u', 'time', 'ur']

In [11]:
sentences = []
for tweet in df['tweet']:
    sentences.append(tweet.split())
    
from gensim.models import Word2Vec
w2vev = Word2Vec(sentences, min_count=5,window=3,size =100)
w2vev.save('w2vec.model')

In [12]:
from gensim.models import Word2Vec
w2vec = Word2Vec.load('w2vec.model')

In [29]:
X = []
for tweet in tqdm(df['tweet']):
    mean = np.zeros(100)
    cnt = 0.0
    for token in tweet.split():
        if token in w2vec.wv.vocab:
            mean += w2vec[token]
            cnt += 1.0
    if cnt!=0: mean /= cnt
    X.append(mean)

100%|██████████| 31931/31931 [00:01<00:00, 22627.13it/s]


31931

In [30]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, df['label'], test_size=0.3, random_state=0)

In [31]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB

gnb = GaussianNB()
gnb.fit(X_train, y_train)
predictions = gnb.predict(X_test)
#################### Gaussian NB Result #########################
print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions))  
print(accuracy_score(y_test, predictions))

[[4258 4626]
 [  84  612]]
              precision    recall  f1-score   support

           0       0.98      0.48      0.64      8884
           1       0.12      0.88      0.21       696

    accuracy                           0.51      9580
   macro avg       0.55      0.68      0.43      9580
weighted avg       0.92      0.51      0.61      9580

0.5083507306889353


In [32]:
from sklearn.ensemble import RandomForestClassifier
text_classifier = RandomForestClassifier(n_estimators=30, random_state=0)  
text_classifier.fit(X_train, y_train)
predictions = text_classifier.predict(X_test)

print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions))  
print(accuracy_score(y_test, predictions))

[[8858   26]
 [ 492  204]]
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      8884
           1       0.89      0.29      0.44       696

    accuracy                           0.95      9580
   macro avg       0.92      0.65      0.71      9580
weighted avg       0.94      0.95      0.93      9580

0.945929018789144
